In [2]:
import os
import torch
import json
from io import BytesIO
import numpy as np
import torch.nn as nn
import imageio.v2 as imageio
from tqdm import tqdm, trange
from utils import PointDataset
from dynamic_model import PointNetModel
import matplotlib.pyplot as plt

In [3]:
device = torch.device('cpu')

In [4]:
data_path = '/Users/reza/Career/DMLab/SURROGATE/Data/psi_web_sample/train'
result_path = '/Users/reza/Career/DMLab/SURROGATE/results/laplace/pointnet_dustin_phi_theta_r/exp_10'
cfg_path = os.path.join(result_path, 'cfg.json')
with open(cfg_path, 'r') as f:
    cfg = json.load(f)

In [5]:
len(cfg['train_files'])

680

In [6]:
instruments = [
        "kpo_mas_mas_std_0101",
        "mdi_mas_mas_std_0101",
        "hmi_mast_mas_std_0101",
        "hmi_mast_mas_std_0201",
        "hmi_masp_mas_std_0201",
        "mdi_mas_mas_std_0201",
    ]
subdir_paths = sorted(os.listdir(data_path))
cr_paths = [os.path.join(data_path, p) for p in subdir_paths if p.startswith("cr")]
sim_paths = []
for cr_path in cr_paths:
    for instrument in instruments:
        instrument_path = os.path.join(cr_path, instrument)
        if os.path.exists(instrument_path):
            sim_paths.append(instrument_path)

In [7]:
dataset = PointDataset(
    sim_paths,
    b_min=cfg["train_min"],
    b_max=cfg["train_max"],
)

In [8]:
cfg['train_min'], cfg['train_max']

(-2.6867804527282715, 2.7947838306427)

In [9]:
model = PointNetModel(dims=cfg['dims'])
state = torch.load(os.path.join(result_path, '7.pth'), map_location=device)
model.load_state_dict(state)

<All keys matched successfully>

In [10]:
x, y = dataset[0]

In [11]:
x.shape

torch.Size([140, 4, 14080])

In [12]:
I = 5
x, y = dataset[I]

model.eval()
with torch.no_grad():
    K = x.shape[0]
    yhats = []
    for k in trange(K):
        xx = x[k, :, :].unsqueeze(0)
        yhat = model(xx.to(device))
        yhats.append(yhat)
    yhats = torch.stack(yhats, dim=1)

100%|██████████| 140/140 [00:13<00:00, 10.61it/s]


In [13]:
y.shape, yhats.shape

(torch.Size([140, 14080, 1]), torch.Size([1, 140, 14080, 1]))

In [14]:
cube = y

In [15]:
frames = []
for i in trange(K):
    y = cube[i, :, :]
    yhat = yhats[:, i, :, :]
    # print(cube.shape)
    y = y.reshape((128, 110))
    # print(yhat.shape)
    yhat = yhat.reshape((128, 110))
    y = y.transpose(1, 0)
    yhat = yhat.transpose(1, 0)

    error = np.abs(y - yhat)
    vmin = min(y.min(), yhat.min())
    vmax = max(y.max(), yhat.max())
    fig, axes = plt.subplots(1, 3, figsize=(12, 6))
    cmap = "coolwarm"

    # # Plot first subplot
    im1 = axes[0].imshow(y, cmap=cmap, vmin=vmin, vmax=vmax)
    axes[0].set_title(f"gt: br002 at {i+1}")

    # Plot second subplot
    im3 = axes[1].imshow(yhat, cmap=cmap, vmin=vmin, vmax=vmax)
    axes[1].set_title(f"pred at {i+1} ")

    cbar = fig.colorbar(im1, ax=axes, orientation="horizontal", fraction=0.1, pad=0.02)

    # Plot second subplot
    im4 = axes[2].imshow(error, cmap="gray")
    axes[2].set_title(f"|pred-gt|")

    # Create a single colorbar
    cbar = fig.colorbar(im4, ax=axes, orientation="vertical", fraction=0.05, pad=0.02)

    # plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format="png")
    frames.append(imageio.imread(buf))
    plt.close()

100%|██████████| 140/140 [00:24<00:00,  5.69it/s]


In [16]:
output_filename = f"b{I}.gif"
fps = 10  # Adjust frames per second as needed

# Create video directly from frames in memory
with imageio.get_writer(output_filename, fps=fps, loop=0) as writer:
    for frame in frames:
        writer.append_data(frame)